In [1]:
!pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters

You should consider upgrading via the '/Users/aryankeshri/Downloads/Entivin_Task/lang_graph_rag/bin/python3 -m pip install --upgrade pip' command.


In [12]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "multihop-rag"

OpenAI API Key········


In [13]:
from langchain.callbacks.tracers import LangChainTracer
# LANGCHAIN_TRACING_V2=True
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="multihop-rag"
tracer = LangChainTracer()
tracer.run_name = "multihop-rag"

In [31]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "/Users/aryankeshri/Downloads/Entivin_Task/book.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [32]:
print(pages)
# print(f"{pages[4].metadata}\n")
# print(pages[4].page_content)

[Document(metadata={'source': '/Users/aryankeshri/Downloads/Entivin_Task/book.pdf', 'page': 0}, page_content='India\nContemporary\nII\nTextbook in Geography\nfor Class X\nSocial Science\nReprint 2024-25\n'), Document(metadata={'source': '/Users/aryankeshri/Downloads/Entivin_Task/book.pdf', 'page': 1}, page_content='Everything available in our envir onment\nwhich can be used to satisfy our needs,\nprovided, it is technologically accessible,\neconomically feasible and culturally\nacceptable can be termed as ‘Resource’.\nThe process of transformation of things\navailable in our environment involves an\ninteractive relationship between nature,\ntechnology and institutions. Human beings\ninteract with nature through technology and\ncreate institutions to accelerate their\neconomic development.\nDo you think that resources are free\ngifts of nature as is assumed by many?\nThey are not. Resources are a function of\nhuman activities. Human beings themselves\nare essential components of resourc

## Create Index

In [33]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import json

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(pages)
print(type(doc_splits))

# Add to vectorDB

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

<class 'list'>


In [35]:
# print(doc)

## LLM

In [36]:
### Retrieval Grader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
# question = "What are Multihop RAG?"
question = "Which country is second largest producer of sugarcane in the world and the larget producer of oilseeds in the world?"
docs = retriever.get_relevant_documents(question)
print(docs)
doc_txt = docs[0].page_content
# doc_txt = doc.context[]
print(doc_txt)
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

[Document(metadata={'page': 36, 'source': '/Users/aryankeshri/Downloads/Entivin_Task/book.pdf'}, page_content='36\n CONTEMPORARY I NDIA – II\nsowing to harvesting. India is the second\nlargest producer of sugarcane only after\nBrazil. It is the main source of sugar , gur\n(jaggary), khandsari and molasses. The major\nsugarcane-producing states are Uttar\nPradesh, Maharashtra, Karnataka, Tamil\nNadu, Andhra Pradesh, T elangana, Bihar ,\nPunjab and Haryana.\nOil Seeds:  In 2018 India was the second\nlargest producer of groundnut in the world\nafter China. In rapeseed production India was\nthird largest producer in the world after\nCanada and China in 2018. Different oil seeds\nare grown covering approximately 12 per cent\nof the total cropped area of the country. Main\noil-seeds produced in India are groundnut,\nmustard, coconut, sesamum (til), soyabean,\ncastor seeds, cotton seeds, linseed and\nsunflower. Most of these are edible and used\nas cooking mediums. However, some of these\nare

In [37]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/Users/aryankeshri/Downloads/Entivin_Task/lang_graph_rag/lib/python3.9/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


India is the second largest producer of sugarcane in the world after Brazil. It is also the largest producer of oilseeds in the world. India produces a variety of oilseeds including groundnut, mustard, coconut, and soyabean.


## Make tool

make tool for your multihop rag here

In [38]:
### Question Decomposer

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Prompt
system = """You are a question decomposer that decomposes a question into 2 or 3 sub-questions, if the original question is complex or indirect or not straightforward.\n
Look at the input and try to reason about the underlying semantic intent / meaning."""

requery_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Decompose into two or three sub-question as per the requirement."
        )
    ]
)

question_decomposer = requery_prompt | llm | StrOutputParser()
question_decomposer.invoke({"question": question})

'1. Which country is the second largest producer of sugarcane in the world?\n2. Which country is the largest producer of oilseeds in the world?'

## Create graphs

Now let's create our graph for implementing multi-hop features

### Define Graph State

In [39]:
from typing import List, Set
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph. 
    
    Attributes:
        question: question
        generation: LLM generation
        iterative_Search: whether to add search
        documents: list of documents
    """
    
    questions: List[str]
    generation: str
    requery_search: str
    documents: List[str]

In [43]:
from langchain.schema import Document
from tqdm.auto import tqdm

def retrieve(state):
    """
    Retrieve documents
    
    Args: 
        state (dict): The current graph state
        
    Returns: 
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    
    print("--Retrieve--\n")
    documents = []
    questions = state['questions']
    
    for question in questions:
        # Retrieval 
        documents.extend(retriever.get_relevant_documents(question))
        
    return {"documents": documents, "questions": questions}

def generate(state):
    """
    Generate Answer: 
    
    Args:
        state (dict): The current graph state
        
    Returns: 
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---Generate---\n")
    
    questions = state['questions']
    documents = state['documents']
    generation = ""
    # RAG generation
    print(documents)
    for question in questions:
        generation = generation + rag_chain.invoke({'context': documents, 'question': question})
        
    return {"documents": documents, "questions": questions[0], "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.
    
    Args: 
        state (dict): The current graph state
    
    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    
    print("---Check Document Relevance to Question---\n")
    question = state["questions"]
    documents = state["documents"]
    print(documents)
    # Score each doc
    filtered_docs = []
    re_query = "No"
    for d in documents: # Its a single loop cause, in the begining there is only one question in the list. and this function is called only in the begining

        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---\n")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---\n")
            re_query = "Yes"
            continue
#         else:
#             print("---No relevant document found---\n")
#             re_query = "Yes"
    return {"documents": filtered_docs, "questions": question, "requery_search": re_query}


def decompose_query(state):
    """
    Decompose the query into sub-queries for better retrieval.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appends multiple questions to questions key with a re-phrased question
    """
    print("---Decompose Query---\n")
    question = state['questions'][0]
    documents = state['documents']
    # Decomposed questions 
    questions_str = question_decomposer.invoke({"question": question})
    print(questions_str)
    print("\n")
    new_questions = questions_str.split('\n')
    questions = state['questions']
    questions.extend(new_questions)
    new_questions = questions
    print(f"new_Questions: {new_questions}")
    return {"documents": documents, "questions": new_questions}

def decide_requery(state):
    """
    Determines whether to generate an answer, or decompose a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """
    print("---Assesing Graded Documents---\n")
    state['questions']
    requery = state['requery_search']
    state['documents']
    
    if requery == "Yes":
        print("---Decomposing Question into sub-questions---\n")
        return "decompose_query"
    else:
        print("---Decision Generate---\n")
        return "generate"

In [44]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("decompose_query", decompose_query)
workflow.add_node("retrieve_after_decomposition", retrieve)
workflow.add_node("generate", generate)
# workflow.add_node("")

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_requery,
    {
        "decompose_query": "decompose_query",
        "generate": "generate"
    },
)
workflow.add_edge("decompose_query", "retrieve_after_decomposition")
workflow.add_edge("retrieve_after_decomposition", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

## Use the Graph

In [49]:
from pprint import pprint

# Run
# inputs = {"questions": ["What is the difference between Atal Bhujal Yojna and Pradhan Mantri Krishi Sinchaee Yojana"]} # comparison question from the same chapter
# inputs = {"questions": ["Which country is second largest producer of sugarcane in the world and the larget producer of oilseeds in the world?"]} # bridge question from the same chapter
inputs = {"questions": ["What is the name of the state having Alluvial Soil and also have oil fields?"]} # bridge question from far apart section/chapters.
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

--Retrieve--

"Node 'retrieve':"
'\n---\n'
---Check Document Relevance to Question---

[Document(metadata={'page': 7, 'source': '/Users/aryankeshri/Downloads/Entivin_Task/book.pdf'}, page_content='Saurashtra, Malwa, Madhya Pradesh and\nChhattisgarh and extend in the south east\ndirection along the Godavari and the Krishna\nvalleys.\nFig. 1.6:  Alluvial Soil\nThe alluvial soil consists of various\nproportions of sand, silt and clay. As we move\ninlands towards the river valleys, soil particles\nappear some what bigger in size. In the upper\nreaches of the river valley i.e. near the place\nof the break of slope, the soils are coarse. Such\nsoils are more common in piedmont plains\nsuch as Duars, Chos and Terai.\nApart from the size of their grains or\ncomponents, soils are also described on the\nbasis of their age. According to their age\nalluvial soils can be classified as old alluvial\n(Bangar ) and new alluvial ( Khadar ). The\nbangar soil has higher concentration of kanker\nThe black

"Node 'generate':"
'\n---\n'
('The state with Alluvial Soil and famous for oil fields is Gujarat. Gujarat '
 'is known for its fertile alluvial soils and major petroleum production areas '
 'like Mumbai High. Additionally, Gujarat has important oil fields like '
 'Ankeleshwar.Madhya Pradesh and Chhattisgarh are known for having Alluvial '
 'Soil.Gujarat is famous for its oil fields, with Mumbai High, Gujarat, and '
 'Assam being major petroleum production areas in India. Assam is the oldest '
 'oil-producing state in India, with important oil fields like Digboi, '
 'Naharkatiya, and Moran-Hugrijan. Oil is also found in fault traps between '
 'porous and non-porous rocks.')
